In [1]:
def update_student(self, student_id, updated_data):
    session = self.Session()
    student = session.query(Student).filter(Student.id == student_id).first()
    if not student:
        session.close()
        return None
    for key, value in updated_data.items():
        setattr(student, key, value)
    session.commit()
    session.refresh(student)
    session.close()
    return student

def delete_student(self, student_id):
    session = self.Session()
    student = session.query(Student).filter(Student.id == student_id).first()
    if not student:
        session.close()
        return None
    session.delete(student)
    session.commit()
    session.close()
    return True


In [5]:

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker
from fastapi import FastAPI, Depends, HTTPException
from pydantic import BaseModel

Base = declarative_base()

class Student(Base):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    last_name = Column(String)
    first_name = Column(String)
    faculty = Column(String)
    course = Column(String)
    grade = Column(Integer)

# Класс менеджера
class StudentManager:
    def __init__(self, db_url='sqlite:///students.db'):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

app = FastAPI()
manager = StudentManager()

def get_db():
    db = manager.Session()
    try:
        yield db
    finally:
        db.close()


In [6]:
class StudentBase(BaseModel):
    last_name: str
    first_name: str
    faculty: str
    course: str
    grade: int

class StudentCreate(StudentBase):
    pass

class StudentUpdate(BaseModel):
    last_name: str = None
    first_name: str = None
    faculty: str = None
    course: str = None
    grade: int = None

class StudentRead(StudentBase):
    id: int
    class Config:
        orm_mode = True


In [7]:
@app.post("/students/", response_model=StudentRead)
def create_student(student: StudentCreate, db=Depends(get_db)):
    data = student.dict()
    new_student = Student(**data)
    db.add(new_student)
    db.commit()
    db.refresh(new_student)
    return new_student

@app.get("/students/{student_id}", response_model=StudentRead)
def read_student(student_id: int, db=Depends(get_db)):
    student = db.query(Student).get(student_id)
    if not student:
        raise HTTPException(status_code=404, detail="Student not found")
    return student

@app.put("/students/{student_id}", response_model=StudentRead)
def update_student(student_id: int, student: StudentUpdate, db=Depends(get_db)):
    db_student = db.query(Student).filter(Student.id == student_id).first()
    if not db_student:
        raise HTTPException(status_code=404, detail="Student not found")
    for key, value in student.dict(exclude_unset=True).items():
        setattr(db_student, key, value)
    db.commit()
    db.refresh(db_student)
    return db_student

@app.delete("/students/{student_id}")
def delete_student(student_id: int, db=Depends(get_db)):
    db_student = db.query(Student).filter(Student.id == student_id).first()
    if not db_student:
        raise HTTPException(status_code=404, detail="Student not found")
    db.delete(db_student)
    db.commit()
    return {"ok": True}
